In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Load data

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Load data
X_full = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv')
X_test_full = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/test.csv')
item_categories = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv')
items = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/items.csv')
X_full = pd.merge(X_full, X_test_full, on=['shop_id', 'item_id'], how='inner', sort=False)
print(X_full.head())
print(items.head())
print(item_categories.head())

In [ ]:
items = pd.merge(items, item_categories, on="item_category_id", how="left")
X_test_full = pd.merge(X_test_full, items, how="left", on="item_id")
X_test_full = X_test_full[["ID", "shop_id", "item_id", "item_category_id"]]
X_test_full

In [ ]:
items = pd.merge(items, item_categories, on="item_category_id", how="left")
X_full = pd.merge(X_full, items, how="left", on="item_id")
X_full

# Prepare data

In [ ]:
X_full = X_full.astype({'ID':'int'})
X_set = X_full.groupby(['ID', 'shop_id', 'item_id', 'item_category_id']).mean().reset_index()
X_set = X_set.rename(columns = {'item_cnt_day': 'item_cnt_month'}, inplace = False)
X_set.drop(['item_price', 'date_block_num'], axis=1, inplace=True)
#X_set['ID'] = X_set.index
#X_group.drop(['month'], axis=1, inplace=True)
X_set

In [ ]:
from sklearn.model_selection import train_test_split

X_set.dropna(axis=0, subset=['item_cnt_month'], inplace=True)
y = X_set['item_cnt_month']
X_set.drop(['item_cnt_month'], axis=1, inplace=True)
#X_full.drop(['date'], axis=1, inplace=True)

  

# Break off validation set from training data
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X_set, y, 
                                                                train_size=.9, test_size=.1,
                                                                random_state=0)
# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if 
                X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
#print(X_test_full)
#X_test = X_test_full[['item_id', 'item_cnt_day']].copy()

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='median')

# Preprocessing for categorical data: No needed

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols)
    ])

# Select a model

In [ ]:
from xgboost import XGBRegressor
model = XGBRegressor(n_estimators=300, learning_rate=0.002, n_jobs=5)

In [ ]:
# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)
                     ])

# Validate model

In [ ]:
my_pipeline.fit(X_train, y_train)
preds = my_pipeline.predict(X_valid)
print('MAE:', mean_absolute_error(y_valid, preds))

# Final training and predictions

In [ ]:
# Preprocessing of training data, fit model 
my_pipeline.fit(pd.concat([X_train, X_valid]), pd.concat([y_train, y_valid]))

In [ ]:
preds = my_pipeline.predict(X_test_full)

# Save predictions to submit

In [ ]:
output = pd.DataFrame({'ID': X_test_full.ID, 'item_cnt_month': preds})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")